<div class="alert">  
    <center><h1><strong>CM-3015 Machine Learning and Neural Networks finals: Training a model using DQN</strong></h1></center>

## Summary

<div class="alert alert-block alert-info">  
    <center><h2><strong>Problem definition and dataset loading</strong></h2></center>

<div class="alert alert-block alert-warning">  
<b>Summary of the section </b> 
<hr>
<ul>
    <li>1. </li>
</ul>
    
<hr>
<b>Source of Data: </b> 
<hr> 
 <a href="https://paperswithcode.com/dataset/liar">https://paperswithcode.com/dataset/liar</a>
   
</div>

In [99]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import reuters
from tensorflow.keras import layers, models
from tensorflow.keras.layers import TextVectorization

# Load the Reuters dataset
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

<div class="alert alert-block alert-info">  
    <center><h2><strong>Preprocessing the dataset</strong></h2></center>

<div class="alert alert-block alert-warning">  
<b>Summary of the section </b> 
<hr>
<ul>
    <ul> Vectorization:
    <li>1.Decodes the numerical sequences back into text using the dataset's word index. </li>
    <li>2.Uses TextVectorization to tokenize and vectorize the dataset into fixed-length sequences.. </li>
    <li>3.Returns vectorized data that can be used in deep learning models. </li>
</ul>
    
<hr>
<b>Source of Data: </b> 
<hr> 
 <a href="https://paperswithcode.com/dataset/liar">https://paperswithcode.com/dataset/liar</a>
   
</div>

In [100]:
# Helper function for vectorizing the data input

# One-hot encode the sequences (vectorizing the input data)
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0  # Set specific indices to 1s
    return results

In [101]:
# One-hot encode the labels
train_data = vectorize_sequences(train_data)
test_data = vectorize_sequences(test_data)

# One-hot encode the output labels (categorical labels)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

<div class="alert alert-block alert-info">  
    <center><h2><strong>Building the model</strong></h2></center>

<div class="alert alert-block alert-warning">  
<b>Summary of the section </b> 
<hr>
Here we will be building our model ...
<hr>


In [102]:
# Helper functions for building the model

from tensorflow.keras import models
from tensorflow.keras import layers

# Define the model
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation="relu", input_shape=(10000,)))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(46, activation="softmax"))  # 46 classes in the dataset
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def print_layer_tensor_shape(layer):
    weight_params = model.layers[layer].get_weights()[0]
    bias_params = model.layers[layer].get_weights()[1]
    print(layer, "\t", weight_params.shape, "\t", bias_params.shape)

In [103]:
# Building the model

model = build_model()
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_56 (Dense)                │ (None, 64)             │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 46)             │         2,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 647,214 (2.47 MB)

 Trainable params: 647,214 (2.47 MB)

 Non-trainable params: 0 (0.00 B)

<div class="alert alert-block alert-info">  
    <center><h2><strong>Training and testing</strong></h2></center>

<div class="alert alert-block alert-warning">  
<b>Summary of the section </b> 
<hr>
Here we will be building our model ...
<hr>


In [104]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import reuters
from tensorflow.keras.utils import to_categorical

model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2991 - loss: 3.4977 - val_accuracy: 0.5797 - val_loss: 2.3267
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6324 - loss: 1.9482 - val_accuracy: 0.6723 - val_loss: 1.5564
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7272 - loss: 1.2974 - val_accuracy: 0.7142 - val_loss: 1.2929
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7857 - loss: 0.9843 - val_accuracy: 0.7556 - val_loss: 1.1566
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8368 - loss: 0.7642 - val_accuracy: 0.7609 - val_loss: 1.0642
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8797 - loss: 0.5882 - val_accuracy: 0.7787 - val_loss: 0.9988
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9015 - loss: 0.4725 - val_accuracy: 0.7787 - val_loss: 0.9650
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9297 - loss: 0.3467 - val_accuracy: 0.7916 - val_loss